This Python script simulates the dynamics of charged particles under relativistic conditions using the Pygame library. It is designed to visualize and explore concepts such as electromagnetic fields, synchrotron radiation, and the effects of relativity on particle interactions. Here's a breakdown of what the script does:

### Imports and Constants
- **Pygame** is used for graphical rendering and interaction.
- **Numpy** is utilized for numerical operations, especially in vector and matrix computations.
- Constants like `WIDTH`, `HEIGHT`, `SPEED_OF_LIGHT`, and `ELEMENTARY_CHARGE` are defined for use in simulations.

### Synchrotron Function
- `synchrotron_function(x)`: Approximates a function related to synchrotron radiation, which particles emit when they are accelerated in magnetic fields.

### Particle Class
- Represents charged particles with properties like position (`x`, `y`, `z`), `charge`, `mass`, and `velocity`.
- `update`: Updates particle's state based on electromagnetic fields and other particles. It calculates forces from given electric and magnetic fields and updates the particle's velocity and position considering relativistic effects (like mass increase at speeds close to the speed of light and time dilation).
- `synchrotron_radiation_spectrum` and `synchrotron_radiation_power`: Calculate the spectrum and power of synchrotron radiation emitted by the particle.
- `retarded_position` and `retarded_fields`: Compute the position and electromagnetic fields of the particle at previous times, accounting for the finite speed of light (this is part of what makes the simulation 'relativistic').
- `relativistic_mass` and `time_dilation`: Compute the relativistic mass and time dilation effects based on the particle’s velocity.

### ParticleManager Class
- Manages a collection of `Particle` instances.
- `add_particle`: Adds a new particle to the simulation.
- `draw`: Draws all particles on the Pygame screen.
- `update_interactions`: Updates interactions between all particles, considering retarded fields, which simulate the delay in the effect of one particle's field on another due to the finite speed of light.
- `update`: Updates all particles’ states and interactions between them.

### Field Functions
- `electric_field` and `magnetic_field`: Functions to compute the electric and magnetic fields at any point in space based on predefined distributions or configurations.

### Simulation Loop
- Initializes Pygame and sets up the simulation environment.
- Enters a loop where it processes user events (like quitting or adding particles with mouse clicks), updates the state of the simulation (particle movements and interactions), and renders the updated state to the screen.
- Uses Pygame’s drawing functions to visualize particles and possibly field lines or intensity.

### Key Concepts Illustrated by the Script
- **Relativistic Effects**: The script accounts for effects such as time dilation and relativistic mass increase, essential when simulating particles moving at speeds close to the speed of light.
- **Electromagnetic Interactions**: It computes interactions between particles based on electromagnetic fields, including those generated by the particles themselves, considering the retardation effect due to the finite speed of light.
- **Synchrotron Radiation**: It calculates and possibly visualizes the radiation emitted by charged particles when accelerated, which is significant in many high-energy physics contexts.

Overall, this script provides a platform for visualizing and understanding the complex dynamics involved in relativistic particle interactions, including electromagnetic forces and radiation. It serves as an educational tool for illustrating concepts from electromagnetism and relativistic physics.

In [ ]:
# V 3.1 Reletivistic particle 
#  
# retarded_electric_field
# synchrotron_radiation

import pygame
import random
import numpy as np
from pygame import surfarray
import colorsys
# Constants
WIDTH, HEIGHT = 800, 600
BLACK = (0, 0, 0)
RED = (255, 0, 0)
SPEED_OF_LIGHT = 299792458  # Speed of light in m/s 
ELEMENTARY_CHARGE = 1.602176634e-19  # Elementary charge in Coulombs
EPSILON_0 = 8.8541878128e-12  # Permittivity of free space in F/m
 
class Particle:
    def __init__(self, x, y, z, charge, mass):
        self.x, self.y, self.z = x, y, z
        self.charge, self.mass = charge, mass
        self.radius = 5  # The radius of the particle
        self.color = (0,0,0)  # Color of the particle
        self.velocity = np.array([0.0, 0.0, 0.0])  # Initialize velocity 
        self.acceleration =  np.array([0.0, 0.0, 0.0])
        self.dt=0.0001
        self.energy_loss_per_second = 0  # Initialize energy loss due to synchrotron radiation

    def update(self, electric_field_function, magnetic_field_function, particles):
        electric_field = electric_field_function(self.x, self.y, self.z) 
        magnetic_field = magnetic_field_function(self.x, self.y, self.z)  
                                           
        # Calculate the retarded fields from all other particles        
        retarded_electric_field, retarded_magnetic_field = self.calculate_retarded_effects(particles)                 
        # Calculate the total electric and magnetic fields
        total_electric_field = electric_field + retarded_electric_field
        total_magnetic_field = magnetic_field + retarded_magnetic_field
         
        self.update_synchrotron_radiation(total_magnetic_field)
        self.update_color_based_on_radiation()
    
        force = self.charge * (total_electric_field + \
                               np.cross(self.velocity, total_magnetic_field))
 
        self.acceleration = force / self.mass
        #self.acceleration = acceleration
        # Update the velocity based on the acceleration
        self.velocity += self.acceleration * self.dt
        # Ensure the velocity does not exceed the speed of light
        speed = np.linalg.norm(self.velocity)
        if speed > SPEED_OF_LIGHT:
            # If the speed exceeds the speed of light, scale the velocity vector down maintains the direction
            self.velocity = (self.velocity / speed) * SPEED_OF_LIGHT

        # Update position based on velocity
        self.x += self.velocity[0] * self.dt
        self.y += self.velocity[1] * self.dt
        self.z += self.velocity[2] * self.dt

        # Wrap around screen, 
        self.x %= WIDTH
        self.y %= HEIGHT                   
        
    def update_synchrotron_radiation(self, magnetic_field_strength):
        # Assuming the magnetic field is perpendicular to the velocity for simplicity
        # Calculate the radius of the particle's path due to the magnetic field
        # Make sure magnetic_field is not zero to avoid division by zero
        magnetic_field_strength = np.linalg.norm(magnetic_field_strength)
        if magnetic_field_strength == 0:
            self.energy_loss_per_second = 0
            return  # Exit the function early if there's no magnetic field
        
        v_mag = np.linalg.norm(self.velocity)
        r = self.mass * v_mag / (np.abs(self.charge) * magnetic_field_strength)

        # Calculate the acceleration
        if r > 0:
            a = v_mag ** 2 / r
        else:
            a = 0  # Set acceleration to zero if radius is zero

        # Calculate the power radiated due to synchrotron radiation
        P = (2.0 / 3) * (self.charge ** 2 * a ** 2) / (4 * np.pi * EPSILON_0 * SPEED_OF_LIGHT ** 3)

        # Convert power loss to energy loss per time step
        self.energy_loss_per_second = P

        # Update the particle's velocity based on the energy loss
        energy = 0.5 * self.mass * v_mag ** 2
        new_energy = energy - self.energy_loss_per_second * self.dt
        if new_energy > 0:
            new_v_mag = np.sqrt(2 * new_energy / self.mass)
            self.velocity *= new_v_mag / v_mag  # Scale velocity to match new energy
        else:
            self.velocity = np.zeros_like(self.velocity)  # Particle stops if energy is depleted
        
    def calculate_retarded_effects(self, particles):
        c = SPEED_OF_LIGHT  # Speed of light in m/s
        epsilon_0 = EPSILON_0  # Vacuum permittivity
        self_position = np.array([self.x, self.y, self.z])
        total_E = np.zeros(3)
        total_B = np.zeros(3)
        for p in particles:
            if p == self:
                continue  # Skip self in the field calculation
            r = self_position - np.array([p.x, p.y, p.z])
            r_mag = np.linalg.norm(r)
            retarded_time = r_mag / c            
            # Assuming constant acceleration for simplicity to calculate retarded position, velocity:
            retarded_position = np.array([p.x, p.y, p.z]) + \
                                            p.velocity * retarded_time + \
                                            0.5 * p.acceleration * retarded_time**2
            retarded_velocity = p.velocity + p.acceleration * retarded_time
            r_retarded = self_position - retarded_position
            r_retarded_mag = np.linalg.norm(r_retarded)
            r_retarded_unit = r_retarded / r_retarded_mag
            # Calculate retarded electric field
            beta = retarded_velocity / c
            gamma = 1 / np.sqrt(1 - np.dot(beta, beta))
            velocity_term = (r_retarded_unit - beta) / (gamma**2 * (1 - np.dot(beta, r_retarded_unit))**3 * r_retarded_mag**2)
            acceleration_term = np.cross(r_retarded_unit, np.cross(r_retarded_unit - beta, p.acceleration / c)) / (c * (1 - np.dot(beta, r_retarded_unit))**3 * r_retarded_mag)
            E = (p.charge / (4 * np.pi * epsilon_0)) * (velocity_term + acceleration_term)
            # Calculate retarded magnetic field
            B = np.cross(r_retarded_unit, E) / c
            total_E += E
            total_B += B

        return total_E, total_B  

    def time_dilation(self):
        # Calculate the time dilation factor        
        speed = np.linalg.norm(self.velocity) 
        return 1 / np.sqrt(1 - (speed / SPEED_OF_LIGHT) ** 2)
    
    def relativistic_energy(self):
        # Calculate the relativistic energy of the particle
        gamma = 1 / self.time_dilation()
        return self.mass * SPEED_OF_LIGHT**2 * gamma         
        
    def update_color_based_on_radiation(self):
        # Define the maximum expected power for scaling purposes
        max_power = 3 #SOME_MAX_POWER_VALUE  # You need to define this based on your simulation's parameters
        # Normalize the radiation power to a 0-1 scale
        normalized_power = min(self.energy_loss_per_second / max_power, 1)                 
        # Hue varies from 0.67 (blue) to 0 (red); adjust if you want different start/end colors
        hue = 0.67 * (1 - normalized_power)
        normalized_speed =  np.linalg.norm(self.velocity) / SPEED_OF_LIGHT 
        saturation = 0.5 + 0.5 * normalized_speed  # Full saturation
        value = 1
        
        # Convert HSV to RGB. colorsys returns floats in the range [0, 1], so convert to [0, 255]
        rgb_float = colorsys.hsv_to_rgb(hue, saturation, value)
        new_color = tuple(int(component * 255) for component in rgb_float)
        self.color = new_color 
        
class ParticleManager:
    def __init__(self):
        self.particles = []  # List to hold all particles

    def update(self, electric_field, magnetic_field):
        for particle in self.particles:
            particle.update(electric_field, magnetic_field, self.particles)
            
    def add_particle(self, x, y, z, charge, mass):
        # Add a new particle to the list
        self.particles.append(Particle(x, y, z, charge, mass))

    def draw(self, screen):
        # Draw all particles
        for particle in self.particles:
            pygame.draw.circle(screen, particle.color, (int(particle.x), int(particle.y)), particle.radius)    
            #particle.draw(screen)         

############################################################################# 
def electric_field(x, y, z):
    # _chaos  Constants
    #k = 8.9875517873681764e9
    k = 1 / (4 * np.pi * EPSILON_0)
    Q = 1
    source_x, source_y, source_z = 400, 300, 0  # Position of the center of the distribution
    sigma_x, sigma_y, sigma_z = 100, 100, 50  # Standard deviations of the distribution
    Lx, Ly = 800, 600  # Dimensions of the simulation domain
    epsilon = 1e-9  # Small value to avoid division by zero

    # Compute differences in coordinates
    dx, dy, dz = x - source_x, y - source_y, z - source_z

    # Compute the Gaussian distribution
    gaussian = Q * np.exp(-0.5 * ((dx/sigma_x)**2 + (dy/sigma_y)**2 + (dz/sigma_z)**2))

    # Compute the chaotic perturbation
    perturbation = 0.1 * (np.sin(2 * np.pi * 2 * x / Lx - z)
                          + 0.5 * np.cos(3 * np.pi * 6 * y / Ly - 2*z)
                          + 0.2 * np.sin(x*y*z))

    # Combine the Gaussian distribution and chaotic perturbation
    field_magnitude = gaussian #* (1 + 2*perturbation)
    #field_magnitude =   perturbation

    # Compute the electric field components
    field_x = k * field_magnitude * dx / (sigma_x**2 )
    field_y = k * field_magnitude * dy / (sigma_y**2 )
    field_z = k * field_magnitude * dz / (sigma_z**2 )

    return np.array([field_x, field_y, field_z])
         
def magnetic_field(x, y, z):
    # Example: a uniform magnetic field in the z direction
    Bz = 300.5
    By = 0
    Bx = 0     
    return np.array([Bx, By, Bz ]) # Bx, By, Bz

def electric_field_spiral (x, y, z):
    # Constants
    k = 8.9875517873681764e9
    Q = 1
    source_x, source_y, source_z = 400, 300, 0  # Position of the center of the distribution
    sigma_x, sigma_y, sigma_z = 50, 50, 50  # Standard deviations of the distribution
    epsilon = 1e-9  # Small value to avoid division by zero

    # Compute differences in coordinates
    dx, dy, dz = x - source_x, y - source_y, z - source_z

    # Compute the radial distance and angle in the xy-plane
    r = np.sqrt(dx**2 + dy**2)
    theta = np.arctan2(dy, dx)

    # Compute the spiral factor
    spiral_factor = np.exp(1j * (r / 20))

    # Compute the Gaussian distribution
    gaussian = Q * np.exp(-0.5 * ((dx/sigma_x)**2 + (dy/sigma_y)**2 + (dz/sigma_z)**2))

    # Compute the electric field components
    field_x = k * gaussian * (dx / (sigma_x**2 + epsilon) * spiral_factor.real - dy / (sigma_y**2 + epsilon) * spiral_factor.imag)
    field_y = k * gaussian * (dy / (sigma_y**2 + epsilon) * spiral_factor.real + dx / (sigma_x**2 + epsilon) * spiral_factor.imag)
    field_z = k * gaussian * dz / (sigma_z**2 + epsilon)

    return np.array([field_x, field_y, field_z])

 
#############################################################################

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Particle Simulation")
clock = pygame.time.Clock()

# Particle manager to manage multiple particles
particle_manager = ParticleManager()
 
particle_manager.add_particle(400, 300, 0, -1.0, 2e7)  #
 
x_grid, y_grid, z_grid = np.meshgrid(range(WIDTH), range(HEIGHT), np.array(0))

electric_fields = electric_field(x_grid, y_grid, z_grid)
magnetic_fields = magnetic_field(x_grid, y_grid, z_grid)

normalized_electric_field =electric_fields# (electric_fields / np.max(np.abs(electric_fields))) * 25500

field_surface = pygame.Surface((WIDTH, HEIGHT))

surfarray.blit_array(field_surface, (normalized_electric_field[0,:,:,0]+normalized_electric_field[1,:,:,0]).reshape(600, 800).T)

running = True
while running:
    for event in pygame.event.get():         
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:  # Check for mouse click
            # Get mouse position and add a new particle there
            x, y = pygame.mouse.get_pos()
            charge = -1# random.choice([-1.0, 1.0])  # Randomly choose charge
            particle_manager.add_particle(x, y,0, charge, 1.0)  # Add new particle at mouse position          
    
    screen.fill(BLACK) 
    screen.blit(field_surface, (0, 0))     
    
    particle_manager.update(electric_field, magnetic_field)
         
    particle_manager.draw(screen)
    pygame.display.flip()
    clock.tick(60)
pygame.quit()